<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD Postprocessing Tutorial
FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).



This notebook will show you the basic postprocessing features that FAST-OAD offers.

**Note: The 01_tutorial notebook has to be ran to generate the results files used in this postprocessing notebook**

*Tip: hover your mouse pointer over any plot to inspect it or get menu to manipulate it*

In [ ]:
import os.path as pth

import fastoad.api as oad
import fastga.utils.postprocessing.post_processing_api as api_plots

# For using all screen width
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

To illustrate the set of postprocessing plots we will use the three designs obtained in the `01_tutorial` notebook:
- Beechcraft 800 nm Multidisciplinary Design Analysis (MDA)
- Beechcraft 1000 nm Multidisciplinary Design Analysis (MDA)
- Beechcraft 800 nm Multidisciplinary Design Optimization (MDO)

In [ ]:
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"

Beechcraft_800nm_MDA_OUTPUT_FILE = pth.join(
    WORK_FOLDER_PATH, "problem_outputs_Beechcraft_800nm_mda.xml"
)
Beechcraft_1000nm_MDA_OUTPUT_FILE = pth.join(
    WORK_FOLDER_PATH, "problem_outputs_Beechcraft_1000nm_mda.xml"
)
Beechcraft_800nm_MDO_OUTPUT_FILE = pth.join(
    WORK_FOLDER_PATH, "problem_outputs_Beechcraft_800nm_mdo.xml"
)

## 1. Introduction

The general philosophy of the postprocessing plots is to use the result data file as the source information. Hence, offline of the computational process. Furthermore, the idea is to compare design results by enabling the superposition of different plots for the postprocessing plots where comparison is relevant.

## 2. Geometry plots
For instance here, we use the `fig` variable generated by the code line 1 and plot the result of line 2 on the existing figure. There is no number of maximum superpositions.

In [ ]:
fig = oad.wing_geometry_plot(Beechcraft_800nm_MDA_OUTPUT_FILE, name="Beechcraft 800 nm MDA")
fig = oad.wing_geometry_plot(
    Beechcraft_1000nm_MDA_OUTPUT_FILE, name="Beechcraft 1000 nm MDA", fig=fig
)
fig = oad.wing_geometry_plot(
    Beechcraft_800nm_MDO_OUTPUT_FILE, name="Beechcraft 800 nm MDO", fig=fig
)
fig.show()

In [ ]:
fig = api_plots.aircraft_geometry_plot(
    Beechcraft_800nm_MDA_OUTPUT_FILE, name="Beechcraft 800 nm MDA"
)
fig = api_plots.aircraft_geometry_plot(
    Beechcraft_1000nm_MDA_OUTPUT_FILE, name="Beechcraft 1000 nm MDA", fig=fig
)
fig = api_plots.aircraft_geometry_plot(
    Beechcraft_800nm_MDO_OUTPUT_FILE, name="Beechcraft 800 nm MDO", fig=fig
)
fig.show()

## 3. Mass breakdown plots

This first mass breakdown plot enables to visualize the Maximum TakeOff Weight (MTOW) and Overall Weight Empty (OWE) for a single design.

In [ ]:
fig = api_plots.mass_breakdown_sun_plot(Beechcraft_800nm_MDA_OUTPUT_FILE)
fig.show()

This second mass breakdown plot provides less detail but enables to compare designs.

## 4. Mission plots

In [ ]:
fig = api_plots.mass_breakdown_bar_plot(
    Beechcraft_800nm_MDA_OUTPUT_FILE, name="Beechcraft 800 nm MDA"
)
fig = api_plots.mass_breakdown_bar_plot(
    Beechcraft_1000nm_MDA_OUTPUT_FILE, name="Beechcraft 1000 nm MDA", fig=fig
)
fig = api_plots.mass_breakdown_bar_plot(
    Beechcraft_800nm_MDO_OUTPUT_FILE, name="Beechcraft 800 nm MDO", fig=fig
)
fig.show()

An other post-processing tool is available: the mission viewer. You simply add the mission files that interest you:

In [ ]:
Beechcraft_800nm_MDA_MISSION_FILE = pth.join(WORK_FOLDER_PATH, "workdir/mda_mission_results.csv")

mission = oad.MissionViewer()
mission.add_mission(Beechcraft_800nm_MDA_MISSION_FILE, name="Beechracft 800 nm MDA")

mission.missions["Beechracft 800 nm MDA"]

Or you can plot the missions:

In [ ]:
mission.display()

## 5. Payload-Range Diagram

FAST-OAD-GA also allows the user to display some interresting plot from an engineering point of view. One such example is the payload-range diagram. Such diagram, though useful for assessing the performances of the aircraft, are not mandatory for the MDA to converge, so they are not included in the base configuration file. This also means that before plotting the diagram we will have to run a special simulation on the file obtained at the output of the MDA before being able to display the actual plot

As we did in the first tutorial, let's set up a run with only the module that computes the payload range diagram (it can take up to 10 min).

In [ ]:
import shutil

CONFIGURATION_FILE_DATA = pth.join(DATA_FOLDER_PATH, "payload_range.yml")
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "payload_range.yml")

SOURCE_FILE_PAYLOAD_RANGE = pth.join(WORK_FOLDER_PATH, "payload_range.xml")

# First copy the configuration file inside the workfolder, and create a duplicate of the MDA outputs

shutil.copy(CONFIGURATION_FILE_DATA, CONFIGURATION_FILE)
shutil.copy(Beechcraft_800nm_MDA_OUTPUT_FILE, SOURCE_FILE_PAYLOAD_RANGE)

# Then generate the inputs

oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE_PAYLOAD_RANGE, overwrite=True)

eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

We will now save the outputs and use the proper post-processing function

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "payload_range_outputs.xml")

fig = api_plots.payload_range(OUTPUT_FILE, name="Beechcraft 800 nm MDA")
fig.show()

## 6. Aircraft polars

Another diagram that we can plot is the aircraft polars (be they equilibrated or not equilibrated). As for the payload-range we first need to launch the analysis before plotting them. This time, it should only take a couple seconds.

In [ ]:
import shutil

CONFIGURATION_FILE_DATA = pth.join(DATA_FOLDER_PATH, "aircraft_polar.yml")
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "aircraft_polar.yml")

SOURCE_FILE_POLAR = pth.join(WORK_FOLDER_PATH, "polar_inputs.xml")

# First copy the configuration file inside the workfolder, and create a duplicate of the MDA outputs

shutil.copy(CONFIGURATION_FILE_DATA, CONFIGURATION_FILE)
shutil.copy(Beechcraft_800nm_MDA_OUTPUT_FILE, SOURCE_FILE_POLAR)

# Then generate the inputs

oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE_POLAR, overwrite=True)

eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

As before we will now save the outputs and use the proper post-processing function as before. Let's start with the non equilibrated one.

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "polar_outputs.xml")

fig = api_plots.aircraft_polar(OUTPUT_FILE, name="Beechcraft 800 nm MDA")
fig.show()

And let's now do the equilibrated one.

In [ ]:
fig = api_plots.aircraft_polar(OUTPUT_FILE, name="Beechcraft 800 nm MDA", equilibrated=True)
fig.show()

## 7. Propeller performances map

It is also possible to display the efficiency maps computed and used in FAST-OAD-CS23. At sea level, it is done like so:

In [ ]:
fig = api_plots.propeller_efficiency_map_plot(Beechcraft_800nm_MDA_OUTPUT_FILE, sea_level=True)
fig.show()

For cruise level, we just need to set the sea_level parameter to `False`

In [ ]:
fig = api_plots.propeller_efficiency_map_plot(Beechcraft_800nm_MDA_OUTPUT_FILE, sea_level=False)
fig.show()

It is also possible to compute and plot the power and thrust coefficient curves. Since it is not done in the sizing process, it has to be computed first. It also requires additional inputs so we won't be able to reuse the output file as is. Instead we will use one that has been prepared for this demonstration.

In [ ]:
CONFIGURATION_FILE_DATA = pth.join(DATA_FOLDER_PATH, "propeller_coefficients.yml")
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "propeller_coefficients.yml")

INPUT_FILE_PROP_DATA = pth.join(DATA_FOLDER_PATH, "propeller_coeff_inputs.xml")
INPUT_FILE_PROP = pth.join(WORK_FOLDER_PATH, "propeller_coeff_inputs.xml")

# First copy the configuration file and the input file inside the workfolder

shutil.copy(CONFIGURATION_FILE_DATA, CONFIGURATION_FILE)
shutil.copy(INPUT_FILE_PROP_DATA, INPUT_FILE_PROP)

eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "propeller_coeff_outputs.xml")

fig = api_plots.propeller_coeff_map_plot(OUTPUT_FILE)
fig.show()